## Prophet Water Level Outliers

Use water level measurements of groundwater available from IntellusNM.com to identify outliers, trend, seasonality, and upset events.

This notebook contains basic statistical analysis and visualization of the data.

### Data Sources
- summary : Processed file from notebook 1-Data_Prep

### Changes
- 02-19-2024 : Started project

In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import seaborn as sns
import prophet

/var/folders/cw/0j2ns2q56n7_rxycqh03plxm0000gn/T/ipykernel_82211/3578604321.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/paulmark/JupyterNotebooks/Water Level Outliers/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
%matplotlib inline

### File Locations

In [3]:
today = datetime.today()
in_file = Path.cwd() / "data" / "processed" / f"summary_{today:%b-%d-%Y}.pkl"
report_dir = Path.cwd() / "reports"
report_file = report_dir / "Excel_Analysis_{today:%b-%d-%Y}.xlsx"

In [4]:
df = pd.read_pickle(in_file)

### Perform Data Analysis

### Save Excel file into reports directory

Save an Excel file with intermediate results into the report directory

In [ ]:
writer = pd.ExcelWriter(report_file, engine='xlsxwriter')

In [ ]:
df.to_excel(writer, sheet_name='Report')

In [ ]:
writer.save()